# Corner detection

Pour détecter les coins des objets dans une image, on peut commencer par déterminer les lieux où deux contours se joignent.
Il existe toutefois d'autres méthodes, parmi lesquelles : 
* le détecteur de Moravec [Moravec 1980],
* le détecteur de Harris [Harris 1988].

## Détecteur de Moravec

À venir...

<!-- 

Le principe de ce détecteur est d'observer les changements d'un patch (une sous-image)
que l'on décale d'un pixel dans toutes les directions, autour d'un pixel d'intérêt.
Si les changements sont importants, alors le patch est centré sur un coin.

```{figure} figs/feature-moravec.png
---
width: 80%
name: F:feature:corners:moravec
---
Principe du détecteur de Moravec.
De gauche à droite :
sur une zone plate, de petits décalages du patch (en rouge) ne provoquent pas de changement ;
sur un contour, on n'observe des changements que dans une seule direction ;
autour d'un coin, il y a des changements significatifs dans toutes les directions.
```

Mathématiquement, le changement est caractérisé par une variable $E$ telle que :

$$
  \forall m,n,x,y \qquad E_{m,n}(x,y) = \sum_{u,v} w_{m,n}(u,v) \big[f(u+x,v+y)-f(u,v)\big]^2
$$

où~:

* $x$ et $y$ représentent les quatre décalages possibles , donc :
  $$
  (x,y) \in \{(1,0),\,(1,1),\,(0,1),\,(-1,1)\}
  $$
sont mesurés en chaque pixel $(m,n)$ de l'image~:

    \item $w_{m,n}$ est une fenêtre rectangulaire autour du pixel ($m,n)$
    \item $\big[f(u+x,v+y)-f(u,v)\big]^2$ représente la différence entre \\
    le patch $f(u,v)$ et le patch décalé $f(u+x,v+y)$
    \item $E_{m,n}(x,y)$ est la différence entre les patchs pour un décalage $(x,y)$
  \end{itemize}


  Le minimum de $E_{m,n}(x,y)$ par rapport aux décalages est déterminé~:
  \begin{equation*}
    \forall m,n \qquad F_{m,n} = \min_{x,y} E_{m,n}(x,y)
  \end{equation*}

  Les coins détectés correspondent aux maxima locaux de $F_{m,n}$.


En chaque pixel $(m,n)$ de l'image est calculé pour différents décalages $(x,y)$~:
\begin{equation*}
  \forall m,n,x,y \qquad E_{m,n}(x,y) = \sum_{u,v} w_{m,n}(u,v) \big[f(u+x,v+y)-f(u,v)\big]^2
\end{equation*}
(seuls $(x,y) \in \{(1,0),\,(1,1),\,(0,1),\,(-1,1)\}$ sont testés).

\bigskip

  où~:
  \begin{itemize}
    \item $w_{m,n}$ est une fenêtre rectangulaire autour du pixel ($m,n)$
    \item $\big[f(u+x,v+y)-f(u,v)\big]^2$ représente la différence entre \\
    le patch $f(u,v)$ et le patch décalé $f(u+x,v+y)$
    \item $E(x,y)$ est la différence entre les patchs pour un décalage $(x,y)$
  \end{itemize}

\end{frame}

\begin{frame}<handout>{Détecteur de Moravec}

En chaque pixel $(m,n)$ de l'image est calculé pour différents décalages $(x,y)$~:
\begin{equation*}
  \forall m,n,x,y \qquad E_{m,n}(x,y) = \sum_{u,v} w_{m,n}(u,v) \big[f(u+x,v+y)-f(u,v)\big]^2
\end{equation*}
(seuls $(x,y) \in \{(1,0),\,(1,1),\,(0,1),\,(-1,1)\}$ sont testés).

\bigskip


  Le minimum de $E_{m,n}(x,y)$ par rapport aux décalages t déterminé~:
  \begin{equation*}
    \forall m,n \qquad F_{m,n} = \min_{x,y} E_{m,n}(x,y)
  \end{equation*}

  Les coins détectés correspondent aux maxima locaux de $F_{m,n}$.


\end{frame}

% =============================================================================================== %

\begin{frame}{Détecteur de Moravec}

\begin{equation*}
  \forall m,n,x,y \qquad E_{m,n}(x,y) = \sum_{u,v} w_{m,n}(u,v) \big[f(u+x,v+y)-f(u,v)\big]^2
\end{equation*}

\bigskip\bigskip

\only<beamer:1|handout:1>{\includegraphics{moravec-example}}
\only<beamer:2|handout:0>{\includegraphics{moravec-example-nocorner}}
\only<beamer:3|handout:0>{\includegraphics{moravec-example-corner}}

\end{frame}

% =============================================================================================== %

\newcommand{\pbnum}[1]{\hspace*{1mm}{\color{unistra}\pscircle[linecolor=unistra](.9ex,.5ex){0.25}\footnotesize P#1}\hspace*{1mm}}

\begin{frame}{Détecteur de Moravec}

Problèmes~:
\begin{enumerate}
  \item[\pbnum{1}] la réponse du détecteur peut être bruitée car $w$ est une fenêtre binaire
  \item[\pbnum{2}] seuls des décalages de 45\degre{} sont considérés
  \item[\pbnum{3}] le détecteur est trop sensible aux contours car seul le minimum de $E$ est considéré
\end{enumerate}

\pause\bigskip

$\Rightarrow$ détecteur de Harris.

\end{frame} -->

## Détecteur de Harris

À venir...

<!-- 

Pour éviter une réponse bruitée (problème \pbnum{1}), la fenêtre rectangulaire $w$
est remplacée par une fenêtre $w$ gaussienne dans~:
\begin{equation*}
  E_{m,n}(x,y) = \sum_{u,v} w_{m,n}(u,v) \left[ f(u+x,v+y) - f(u,v) \right]^2
\end{equation*}

\end{frame}

% =============================================================================================== %

\begin{frame}{Détecteur de Harris}

Pour étendre le modèle à n'importe quel angle (problème \pbnum{2}),
on utilise un développement en série de Taylor de $f(u+x,v+y)$~:

\begin{equation*}
  f(u+x,v+y) \approx f(u,v) + x \,\partial_x f(u,v) + y \,\partial_y f(u,v)
\end{equation*}
\vspace*{-\baselineskip}\pause
\begin{align*}
  \Rightarrow\quad
  E_{m,n}(x,y) &= \sum_{u,v} w_{m,n}(u,v) \left( f(u+x,v+y) - f(u,v) \right)^2 \\
         &\approx \sum_{u,v} w_{m,n}(u,v) \left( x \,\partial_x f(u,v) + y \,\partial_y f(u,v) \right)^2 \\
         &\approx
         \begin{pmatrix} x & y \end{pmatrix}
         M
         \begin{pmatrix} x \\ y \end{pmatrix}
\end{align*}
\begin{equation*}
  \text{où}\qquad
  M = \sum_{u,v} w_{m,n}(u,v)
  \begin{pmatrix}
    (\partial_x f)^2  &  \partial_x f \,\partial_y f \\
    \partial_x f \,\partial_y f  &  (\partial_y f)^2 \\
  \end{pmatrix}
\end{equation*}

\end{frame}

% =============================================================================================== %

\begin{frame}{Détecteur de Harris}

Le problème \pbnum{3} peut être évité en considérant une nouvelle mesure de la présence d'un coin~:
on peut obtenir d'autres informations sur le changement d'intensité dans la fenêtre
en analysant les valeurs propres $\lambda_1$ et $\lambda_2$ de la matrice $M$.

\bigskip

\begin{minipage}{.49\textwidth}
  \centering$\displaystyle E(x,y) = \begin{pmatrix} x & y \end{pmatrix} M \begin{pmatrix} x \\ y \end{pmatrix}$
\end{minipage}
\begin{minipage}{.49\textwidth}
  \centering\includegraphics{harris-eigenvector}
\end{minipage}

\end{frame}

% TODO : images telles que :
% /home/vincent/Ecole/OFTI/S3 Détection de caractéristiques + recalage/caracteristiques/figs/toto-16.png
% /home/vincent/Ecole/OFTI/S3 Détection de caractéristiques + recalage/caracteristiques/figs/toto-17.png

% =============================================================================================== %

\begin{frame}{Détecteur de Harris}

Le calcul des valeurs propres de $M$ pouvant être difficile,
une alternative est de calculer~:
\begin{equation*}
  R = \mathrm{det}(M) - k (\mathrm{trace}(M))^2
    = \lambda_1 \lambda_2 - k(\lambda_1 + \lambda_2)^2
\end{equation*}
avec $0,04 < k < 0,06$.

\pause

\begin{minipage}{.52\textwidth}
Les valeurs de $R$ sont~:
\begin{itemize}
  \item faibles dans une région plate,
  \item négatives sur un contour,
  \item positives sur un coin.
\end{itemize}
\end{minipage}
\begin{minipage}{.46\textwidth}
  \centering\includegraphics{harris-eigenvector-R}
\end{minipage}

\end{frame}

% =============================================================================================== %

\begin{frame}{Détecteur de Harris}

\begin{center}
  \includegraphics[width=.3\textwidth]{moulinsart-harris-edge}\quad
    \includegraphics[width=.3\textwidth]{moulinsart-harris-const}\quad
  \includegraphics[width=.3\textwidth]{moulinsart-harris-corner}\\
  \parbox{.3\textwidth}{\centering $R<-10^8$}\quad
  \parbox{.3\textwidth}{\centering $|R|<10^8$}\quad
  \parbox{.3\textwidth}{\centering $R>10^8$}\\
\end{center}

% \begin{center}\includegraphics[width=10cm]{toto-22}\end{center}
% \hspace*{4.8cm}\parbox{6.2cm}{\centering $R$ \\ (après un filtre de Sobel)}
% \end{frame}
%
% \begin{frame}{Détecteur de Harris}
% \begin{center}\includegraphics[width=10cm]{toto-25}\end{center}
% \hspace*{4.8cm}\parbox{6.2cm}{\centering $R\in[-10^4, \, 10^4]$ $\Rightarrow$ zones constantes}
% \end{frame}
%
% \begin{frame}{Détecteur de Harris}
% \begin{center}\includegraphics[width=10cm]{toto-23}\end{center}
% \hspace*{4.8cm}\parbox{6.2cm}{\centering $R<-10^4$ $\Rightarrow$ contours}
% \end{frame}
%
% \begin{frame}{Détecteur de Harris}
% \begin{center}\includegraphics[width=10cm]{toto-24}\end{center}
% \hspace*{4.8cm}\parbox{6.2cm}{\centering $R>10^4$ $\Rightarrow$ coins}
\end{frame}

% =============================================================================================== %
 -->